# Top 10 Predictions

A binary problem. 

Every race result since 1995 should give us labels and some important index, like driver, raceId and circuit name. These indices can be used to join other information

Following information on driver:
* Previous year race result on that track (if that circuit was not raced on last year, then value will be -1)
* Qualifying result for that race and driver
* Qualifying timing (in seconds) for that driver in that race
* Practice 1 and Practice 2 result for that race and driver
* Best timing for that driver in Practice Session 1 and 2
* (Optional, if model is not good enough) Telemetry information in qualifying and practice sessions

Following information on circuit:
* Winning driver's average laptime last year
* Number of turns
* Type of circuit (street or purpose built)
* Track temperature
* Air temperature

# Package and Data Imports

In [1]:
# Standard library imports
from ast import literal_eval
import os
import sys

# Third-party imports
import numpy as np
import pandas as pd

# Local application imports
module_path = os.path.abspath(os.path.join("..", "scripts"))
if module_path not in sys.path:
    sys.path.append(module_path)
from utilities import *

In [ ]:
# This dataframe will be used for building a model
final_data = pd.DataFrame()

In [ ]:
# Read race results
race_results = pd.read_csv("../data/raw/Race_Results_1995_2023.csv")
race_results = remove_unnamed_col(race_results)
display(race_results.head())

# Read qualification results
qualification_results = pd.read_csv("../data/raw/Qualification_Results_1995_2023.csv")
qualification_results = remove_unnamed_col(qualification_results)
display(qualification_results.head())

In [ ]:
drivers_info = pd.read_csv("../data/raw/drivers_information.csv")
drivers_info = remove_unnamed_col(drivers_info)
drivers_info

In [ ]:
# Create a function to get driverId given name

def get_driver_id(name):
    """function to get driverId given name"""
    split_name = name.split(" ")
    # If name is in format "Lewis Hamilton return lewis_hamilton"
    if len(split_name) == 2:
        return "_".join(split_name)
    elif len(split_name) == 1:
        return split_name[0]


In [ ]:
# Season and Round will help recognise the race and circuit
final_data["Season"] = race_results["season"] # Season number
final_data["Round"] = race_results["round"] # Round number

# Position can be changed into labels of 0 and 1 (1 if position is in top10)
final_data['Position'] = race_results["position"]
final_data["Label"] = final_data['Position'].apply(lambda x: 1 if x <= 10 else 0)

# Join qualification performance
# TODO: